Imports

In [2]:
import psycopg2
from config import config

Define connect method to connect to the postgres database and return the connection

In [3]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        return conn

        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

Connect to the database

In [4]:
conn = connect()

Connecting to the PostgreSQL database...


Test Query

In [11]:
cur = conn.cursor()

cur.execute("CREATE TABLE test (a int, b int)")

cur.execute("INSERT INTO test VALUES (1,2)")
cur.execute("INSERT INTO test VALUES (4,5)")

cur.execute("SELECT * FROM test")

print(cur.fetchall())

cur.close()

[(1, 2), (4, 5)]


Undo all that stuff

In [10]:
cur = conn.cursor()

cur.execute("DROP TABLE test")

cur.close()